In [1]:
## Importing Necessary Libraries
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
import spacy

/home/kha060/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2025-10-16 12:18:35.177385: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 12:18:35.202134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register facto

In [ ]:
## If you need to install, uncomment it
#pip install transformers==4.35.2
#!pip install tokenizers==0.15.2
# !pip uninstall -y protobuf
# !pip install protobuf==4.25.3

In [2]:
preprocessed_df = pd.read_json('../dataset/ModulesIssuesandPRs/preprocessed_df_dataset_of_nfcore_modules.json')
preprocessed_df

,id,title,text,created_at,closed_at,state
36,3469931397,POC: topics,POC topic serve example work do,2025-09-30 15:03:07,NaT,open
37,3469216905,Rastair fix,Rastair fix nfcoremodule many thank contrib...,2025-09-30 12:37:38,2025-09-30 13:12:09,closed
38,3468699233,new subworkflow: fasta_vclust_prefilter_align_...,new subworkflow fastavclustprefilteralignclust...,2025-09-30 10:30:27,2025-09-30 12:25:42,closed
40,3468384531,[FEATURE] PureCN: support VCF input as part of...,feature purecn support VCF input part purecnru...,2025-09-30 09:21:17,2025-10-14 11:58:19,closed
41,3465159904,Sort inputs to sawfish joint-call to create re...,sort inputs sawfish jointcall create reproduci...,2025-09-29 13:57:44,2025-09-29 14:49:25,closed
...,...,...,...,...,...,...
9176,474621503,Replace Slack join URL,replace Slack join url,2019-07-30 14:32:09,2019-07-30 14:33:42,closed
9177,473715069,Module tests,module test various test perform individual mo...,2019-07-28 07:30:34,2021-03-26 00:13:04,closed
9178,473714655,Use remote repos with include statement,use remote repos include statement need test p...,2019-07-28 07:24:41,2020-07-11 06:46:43,closed
9179,473714364,Module file versioning,module file versione need come way version mod...,2019-07-28 07:20:50,2020-07-10 10:00:33,closed


In [3]:
from sentence_transformers import SentenceTransformer
## Precalculate Embedding
embedding_model = SentenceTransformer("all-mpnet-base-v2")


/home/kha060/anaconda3/envs/BTI/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/kha060/anaconda3/envs/BTI/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/kha060/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
## Removing Stop Words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
## Setting SWS name as stop words so that it can not make impact on the topic modeling
additional_stopwords=["prepare","merge","merged","release","build","repo","code", "nfcore","nfcore","release","pr", "request", "dev", "checklist","soon", "lint", "bump", "thank", "page", "citation", "contributor", "href", "sure",'ensure', 'suite', 'pass', 'authorscontributor', 'unless', 'remember', 'contributingmd', 'pullrequest',  'important', 'possible', 'line', 'like']
stop_words.update(additional_stopwords)

## Function to remove stopwords
## Remove stop words
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

preprocessed_df['text'] = preprocessed_df['text'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /home/kha060/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
preprocessed_df['text'] = preprocessed_df['text'].astype(str)
# Try encoding
try:
    embeddings_Text = embedding_model.encode(preprocessed_df['text'].tolist(), show_progress_bar=True)
except KeyError as e:
    print(f"KeyError encountered: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Batches:   0%|          | 0/237 [00:00<?, ?it/s]

In [26]:
## Clustering
from umap import UMAP
umap_model = UMAP(n_neighbors=20, n_components=25, min_dist=0.1, random_state=42, metric='cosine')

from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=90, metric='euclidean', min_samples=100, cluster_selection_method="leaf", prediction_data=True)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer


from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired, PartOfSpeech
representation_model = MaximalMarginalRelevance(diversity=0.8)


vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1,2))


In [28]:
from bertopic import BERTopic

topic_model = BERTopic(
    embedding_model = embedding_model,
    umap_model = umap_model,
    representation_model=representation_model,
    hdbscan_model = hdbscan_model,
    vectorizer_model = vectorizer_model,
    language = "english",
    verbose = True
)

## Topics Generation
topics, probs = topic_model.fit_transform(preprocessed_df['text'], embeddings_Text)
topic_model.get_topic_info()

2025-10-16 12:41:35,373 - BERTopic - Reduced dimensionality
2025-10-16 12:41:35,991 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name,Representation,Representative_Docs
0,-1,4218,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...
1,0,823,0_open_module search_exist issue_work planning,"[open, module search, exist issue, work planni...",[new module taxonkitreformat exist module x se...
2,1,591,1_convention_add test_fix_module,"[convention, add test, fix, module, option, em...",[hhsearch comment contain description change r...
3,2,416,2_follow_contribute_convention_add test,"[follow, contribute, convention, add test, fix...",[fixmetalinite nfcoremodule many contribute nf...
4,3,305,3_6226_add nftest_update_cooler,"[6226, add nftest, update, cooler, swap, picar...","[add nft serobarun add nft serobarun 6226, add..."
5,4,289,4_drpatelh_output_website_description bug,"[drpatelh, output, website, description bug, m...",[bammarkduplicatespicard incorrect take doc ch...
6,5,216,5_nft migration_pilon_replace_ggetgget,"[nft migration, pilon, replace, ggetgget, yara...","[nft migration elprepmerge, nft migration varl..."
7,6,190,6_test_follow_requirement_module,"[test, follow, requirement, module, contribute...",[add seuratmultiseq nfcoremodule many contribu...
8,7,173,7_test_module_follow_genome,"[test, module, follow, genome, tool, datum, pa...",[nft update genotypegvcfs nfcoremodule many co...
9,8,138,8_new module_open_text link_understandable,"[new module, open, text link, understandable, ...",[new module kaijukaiju2table nfcoremodule new ...


In [29]:
topic_model.get_topic_info().to_csv('../dataset/RQ1/topics_info-nf-core-modules.csv')

In [30]:
# Document Distribution
document_distribution = topic_model.get_document_info(preprocessed_df['text'])
document_distribution

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,POC topic serve example work,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
1,Rastair fix nfcoremodule many contribute nfcor...,2,2_follow_contribute_convention_add test,"[follow, contribute, convention, add test, fix...",[fixmetalinite nfcoremodule many contribute nf...,follow - contribute - convention - add test - ...,1.000000,False
2,new subworkflow fastavclustprefilteralignclust...,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
3,feature purecn support VCF input part purecnru...,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
4,sort inputs sawfish jointcall create reproduci...,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
...,...,...,...,...,...,...,...,...
7553,replace Slack join url,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
7554,module test various test perform individual mo...,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
7555,use remote repos include statement need test p...,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
7556,module file versione need come way version mod...,-1,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False


In [31]:
concatenated_df = pd.concat([preprocessed_df, document_distribution], axis=1)
concatenated_df = concatenated_df[concatenated_df['Topic'].notna()]
concatenated_df

,id,title,text,created_at,closed_at,state,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
36,3.469931e+09,POC: topics,POC topic serve example work,2025-09-30 15:03:07,NaT,open,snpeff snpsift nfcoremodule many contribute nf...,2.0,2_follow_contribute_convention_add test,"[follow, contribute, convention, add test, fix...",[fixmetalinite nfcoremodule many contribute nf...,follow - contribute - convention - add test - ...,0.923195,False
37,3.469217e+09,Rastair fix,Rastair fix nfcoremodule many contribute nfcor...,2025-09-30 12:37:38,2025-09-30 13:12:09,closed,update concoct use late theoretically right ve...,1.0,1_convention_add test_fix_module,"[convention, add test, fix, module, option, em...",[hhsearch comment contain description change r...,convention - add test - fix - module - option ...,1.000000,False
38,3.468699e+09,new subworkflow: fasta_vclust_prefilter_align_...,new subworkflow fastavclustprefilteralignclust...,2025-09-30 10:30:27,2025-09-30 12:25:42,closed,Decoupler nfcoremodule many contribute nfcorem...,-1.0,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
40,3.468385e+09,[FEATURE] PureCN: support VCF input as part of...,feature purecn support VCF input part purecnru...,2025-09-30 09:21:17,2025-10-14 11:58:19,closed,update module mobster exist module x search ex...,0.0,0_open_module search_exist issue_work planning,"[open, module search, exist issue, work planni...",[new module taxonkitreformat exist module x se...,open - module search - exist issue - work plan...,0.344491,False
41,3.465160e+09,Sort inputs to sawfish joint-call to create re...,sort inputs sawfish jointcall create reproduci...,2025-09-29 13:57:44,2025-09-29 14:49:25,closed,deprecate mashdb argument gtdbtkclassifywf inp...,1.0,1_convention_add test_fix_module,"[convention, add test, fix, module, option, em...",[hhsearch comment contain description change r...,convention - add test - fix - module - option ...,0.725593,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7537,NaN,NaN,NaN,NaT,NaT,NaN,add FastQ Screen module need work test workflow,-1.0,-1_test_add_convention_nfcoremodule,"[test, add, convention, nfcoremodule, make, do...",[new module deepvariantvcfstatsreport nfcoremo...,test - add - convention - nfcoremodule - make ...,0.000000,False
7541,NaN,NaN,NaN,NaT,NaT,NaN,port module singletool workflow Babraham work ...,7.0,7_test_module_follow_genome,"[test, module, follow, genome, tool, datum, pa...",[nft update genotypegvcfs nfcoremodule many co...,test - module - follow - genome - tool - datum...,1.000000,False
7543,NaN,NaN,NaN,NaT,NaT,NaN,new tool shovill bacterial assembly Write modu...,7.0,7_test_module_follow_genome,"[test, module, follow, genome, tool, datum, pa...",[nft update genotypegvcfs nfcoremodule many co...,test - module - follow - genome - tool - datum...,1.000000,False
7548,NaN,NaN,NaN,NaT,NaT,NaN,fix trim galore process name,4.0,4_drpatelh_output_website_description bug,"[drpatelh, output, website, description bug, m...",[bammarkduplicatespicard incorrect take doc ch...,drpatelh - output - website - description bug ...,0.931227,False


In [33]:
distinct_topics = concatenated_df['Topic'].unique()

for topic in distinct_topics:
    df_topic = concatenated_df[concatenated_df['Topic'] == topic]
    filename = f"{topic}_Topics_data.csv"
    df_topic.to_csv('../dataset/RQ1/Topic Distribution/' + filename)